## 1. Le modèle 

### 1.1 Principe

La régression logistique est un méthode d'apprentissage supervisée très utile pour expliquer / prédire une variable discrète (à l'opposé de continu) lorsque le jeu de données comporte de nombreuses variables catégorielles.

### 1.2 Notations

Soit Y la variable à prédire / expliquée (dans notre cas, il s'agit de la souscription ou non à l'option de deposit à long terme) et X = (X1, X2, ..., Xn) les variables explicatives

Dans le cadre de la regression logistique binaire, la variable Y prend deux modalités {0,1}. Les variables $X_{i}$ sont exclusivement binaires ou continues.

- Soit $\Omega$ un ensemble de n échantillons (ici n = 45211) comportant $n_{1}$ (respectivement $n_{0}$)) observations correspondant à la modalité 1 (respectivement 0) de Y (dans notre cas, $n_{1}$ = 5289 et $n_{0}$ = 39922).
- $P(Y=1)$ est la probabilité a priori pour que Y = 1. Pour simplifier on la notera p(1)
- $p(X \vert 1)$ est la distribution conditionnelle des X sachant la valeur prise par Y. 
- La probabilité a posteriori d'obtenir la modalité 1 de Y (resp. 0) sachant la valeur prise par X est $p(1 \vert X)$ (resp. $p(0 \vert X)$)


### 1.3 Le modèle LOGIT

On désigne par le terme LOGIT de $p(1 \vert X)$ l'expression suivante: $\ln {\frac  {p(1\vert X)}{1-p(1\vert X)}}=b_{0}+b_{1}x_{1}+...+b_{n}x_{n}$

- il s'agit bien d'une regression car on cherche à montrer une relation de dépendance entre une variable à expliquer telle une série de variables explicatives
- il s'agit d'une regression logistique car la loi de probabilité est modélisée par une loi logistique (https://fr.wikipedia.org/wiki/Fonction_logistique_(Verhulst))

#### Q.1 Pourquoi la fonction logistique est-elle intéressante ? Notamment par rapport à une fonction affine 

Réponse: 

Parmi de nombreux algorithmes de classification d'apprentissage automatique, la régression logistique est l'un des plus largement utilisés et des plus populaires. Il peut être utilisé dans les problèmes de classification binaire et multi-classes. 


f (x) est le Fonction sigmoïde

Passons maintenant à l'hypothèse de la régression logistique:

La sortie de la fonction Sigmoid va de 0 à 1. Mais pour la classification, il faut prédire une étiquette particulière qui indique une classe. Dans ce cas, un seuil (évidemment, une valeur comprise entre 0 et 1) doit être défini de manière à obtenir la performance prédictive maximale (l’analyse de la performance est traitée dans la dernière partie de l’article).

Expliquer / prédire une variable binaire consiste alors à estimer au mieux les coefficients $b_{i}$ ci-dessus. Ensuite on peut retrouver la probabilité conditionnelle par la formule suivante: 

$p(1\vert X)={\frac  {e^{{b_{0}+b_{1}x_{1}+...+b_{n}x_{n}}}}{1+e^{{b_{0}+b_{1}x_{1}+...+b_{n}x_{n}}}}}$

Il existe plusieurs méthodes de resolutions / d'estimations de ces paramètres (voir principe du maximum de vraisemblance et algo méthode de résolution numérique Newton-Raphson). Nous allons nous concentrer plus sur l'interprétation des résultats dans le cas d'une classification plutot que sur les méthodes de résolution.

## 2. Implémentation

### 2.1 Import des librairies et du jeu de données

In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn import linear_model

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

On réutilise le jeu de données sur le marketing dans la banque

In [4]:
df = pd.read_csv("bank-full.csv",  sep = ",")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


### 2.2 Preprocessing des données (voir le package en détails)

Comme précisé plus haut, on a besoin de variables exclusivement binaires ou continues. 

#### Q.2 Quel type d'encoding des variables catégorielles doit-on mettre en place pour ce faire ? 

#### Consigne 1: Mettre en place cet encoding 

Vous pouvez faire cet encoding directement via des instructions dans le notebook. Mais le mieux étant de créer un package dans un script python en dehors du notebook pour faire ce processing et appeler ensuite ce package dans le notebook (comme si vous appeliez un package de pandas ou numpy)

Réponse: 

In [5]:
df_final= df
dummies = pd.get_dummies(df_final["job"])
df_final= pd.concat([df_final, dummies], axis=1)
df_final.head()

dummies = pd.get_dummies(df_final["month"])
df_final= pd.concat([df_final, dummies], axis=1)

dummies = pd.get_dummies(df_final["marital"])
df_final= pd.concat([df_final, dummies], axis=1)

dummies = pd.get_dummies(df_final["education"])
df_final= pd.concat([df_final, dummies], axis=1)

dummies = pd.get_dummies(df_final["default"])
df_final= pd.concat([df_final, dummies], axis=1)

dummies = pd.get_dummies(df_final["housing"])
df_final= pd.concat([df_final, dummies], axis=1)

dummies = pd.get_dummies(df_final["contact"])
df_final= pd.concat([df_final, dummies], axis=1)

dummies = pd.get_dummies(df_final["poutcome"])
df_final= pd.concat([df_final, dummies], axis=1)

dummies = pd.get_dummies(df_final["y"],prefix='y', drop_first=True)
df_final= pd.concat([df_final, dummies], axis=1)


df_final.columns
df_final

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,admin.,blue-collar,entrepreneur,housemaid,management,retired,self-employed,services,student,technician,unemployed,unknown,apr,aug,dec,feb,jan,jul,jun,mar,may,nov,oct,sep,divorced,married,single,primary,secondary,tertiary,unknown,no,yes,no,yes,cellular,telephone,unknown,failure,other,success,unknown,y_yes
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1,1
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,1,1
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0


#### Consigne 2: Séparer l'échantillon de données en base d'apprentissage de base de test (70% / 30%)

Nous scindons la base fournie par bank.csv en deux parties :

Train (données d'apprentissage): Via un ensemble fini de clients, nous allons pouvoir inférer le score d'apétence de la manière la plus générale possible : on part des données pour extraire des règles générales. Cette partie nous permettra de décomposer la matrice de Y en un produit de matrices clients et Y (oui/non).

Test (données de test): Nous allons ensuite tester la fonction apprise sur de nouvelles données, le but étant d'avoir le moins d'erreur possible.

On considere la dataset en :
    - 2/3 apprentissage
    - 1/3 test
    
    ==> qu'on appelera proportion t

In [6]:
import random
def generate_dataset(n, t):
    return [1 if random.random() < t else 0 for i in range(0,n)]

ens = generate_dataset(4000, 0.01)
sum(ens)

39

Et on divise en base d’apprentissage et de test.

In [7]:
def custom_split_train_test(ens, p):
    choice = generate_dataset(len(ens), p)
    train = [x for x, c in zip(ens, choice) if c == 1]
    test = [x for x, c in zip(ens, choice) if c == 0]
    return train, test

train, test = custom_split_train_test(ens, 0.66)
print(len(train), sum(train), sum(train)/len(train))
print(len(test), sum(test), sum(test)/len(test))

2614 27 0.010328997704667177
1386 12 0.008658008658008658


#### On recommence un grand nombre de fois et on représente la proportion obtenue dans la base de test.

In [8]:
tirages = [sum(test)/len(test) for train, test in [custom_split_train_test(ens, 0.66) for i in range(0,100)]]

#### Repartition Stratifiée

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_final, y, test_size=0.3)


NameError: name 'y' is not defined

In [10]:
df_final.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y', 'admin.', 'blue-collar', 'entrepreneur',
       'housemaid', 'management', 'retired', 'self-employed', 'services',
       'student', 'technician', 'unemployed', 'unknown', 'apr', 'aug', 'dec',
       'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep',
       'divorced', 'married', 'single', 'primary', 'secondary', 'tertiary',
       'unknown', 'no', 'yes', 'no', 'yes', 'cellular', 'telephone', 'unknown',
       'failure', 'other', 'success', 'unknown', 'y_yes'],
      dtype='object')

In [1]:
# create training and testing vars

y = df_final.y_yes
X_train, X_test, y_train, y_test = train_test_split(df_final, y, test_size=0.3)

#X_train,X_test,y_train,y_test=train_test_split(new_df,y_yes,test_size=0.30,random_state=0)

NameError: name 'df_final' is not defined

#### Consigne 3: Utiliser la fonction de scikit learn permettant de mettre en place une régression logistique pour entrainer un modèle 

In [17]:
# fit a model

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()


logreg.fit(X_train,y_train)

C:\Users\utilisateur\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: 'no'

In [14]:
X = df_final.drop('y', axis=1)
y = df_final['y']

#### Consigne 4: Faire des prédictions sur l'échantillon de test

In [16]:

predictions[0:5]

NotFittedError: This LogisticRegression instance is not fitted yet

#### Consigne 5: Créez une matrice de confusion pour analyser les résultats

In [20]:
from sklearn import metrics
predictions = logreg.predict(X_test)
cnf_matrix = metrics.confusion_matrix(y_test, predictions)

NotFittedError: This LogisticRegression instance is not fitted yet

#### Q.3: Combien a-t'on de TP (True Positif), TN (True Negatif), FP (False Positive) et FN (False Negative) ? 

#### Q.4 Quelle est l'accuracy du modèle ? Sa précision ? Son recall ? Son F-1 score ?

#### Consigne 6: Construire une courbe ROC et une courbe Precision - Recall à partir des résultats

#### Consigne 7: Faire des prédictions sur l'échantillon d'apprentissage et calculer toutes les métriques ci-dessus

#### Consigne 8: Comparer avec les résultats des prédictions sur l'échantillon d'apprentissage et l'échantillon de test 

#### Q.5 Observe t'on un phénomène particulier ? 

#### Bonus: modifier les hyperparamètres du modèle pour essayer d'obtenir de meilleurs résultats

In [47]:
import random
def generate_dataset(n, t):
    return [1 if random.random() < t else 0 for i in range(0,n)]

ens = generate_dataset(4000, 0.01)
sum(ens)import random
def generate_dataset(n, t):
    return [1 if random.random() < t else 0 for i in range(0,n)]

ens = generate_dataset(4000, 0.01)
sum(ens)

SyntaxError: invalid syntax (<ipython-input-47-4b0a1c781b42>, line 6)